In [1]:
%reload_ext jetisu.query_idr_magic

# Birthday Money

## Source

| Key               | Value                                                                                                   |
|-------------------|---------------------------------------------------------------------------------------------------------|
| Author            | Ram Parameswaran                                                                                        |
| Link              | [![Birthday Monday](https://img.youtube.com/vi/qnn8p8xaP1U/0.jpg)](https://youtu.be/qnn8p8xaP1U?t=3999) |
| Original Language | OpenFisca                                                                                               |
| Retrieved         | 2022-Nov-27                                                                                             |

_Lets revisit the example above using sigma complete relations (IDRs)..._

## Initially a flat rate that applies to all children

Initially the policy for birthday money was calculated as $10.00 per year. Here is a sigma complete relation:
```standard_birthday_money``` that captures this.

In [2]:
%%jetisu_show
standard_birthday_money

```

predicate standard_birthday_money(
        var int: age,
        var float: birthday_money) =
let {
    constraint birthday_money = age*10.00; % constraints express a relationship
} in true;
```

The language used to define sigma complete relations is not so important, the crucial aspect is that the reader, and the computing system, understand the predicate.  The
definition above happens to have been written in [MiniZinc](https://www.minizinc.org/).

Sigma complete relations can be queried as if they are an ordinary database table that contains _all_ the answers. With this table defined, the
following query gives the expected answer:

In [3]:
%%jetisu_query
select * from standard_birthday_money where age = 10;

|age|birthday_money|
|----|----|
|10.0|100.0|

And Query 2 also gives the same answer!

In [4]:
%%jetisu_query
select * from standard_birthday_money where birthday_money=100;

|age|birthday_money|
|----|----|
|10|100.0|

Notice that the relationship can be queried from multiple directions.
## Made conditional on behaviour

At some point of time, the amount of birthday money was made contingent on the child's behaviour. Here is the second version.


In [5]:
%%jetisu_show
new_birthday_money

```

predicate new_birthday_money(
    var int: age,
    var bool: good_behaviour,
    var float: birthday_money) =
let {
    constraint birthday_money = if good_behaviour then age*20.00 else 0.00 endif
} in true;
```

If we now repeat ```Query 1``` we get a different output.

In [6]:
%%jetisu_query
select * from new_birthday_money where age = 10 ;

|age|good_behaviour|birthday_money|
|----|----|----|
|10.0|False|0|
|10.0|True|200|

This may be read as "a child turning ten will receive either nothing, or two hundred dollars birthday money"

As soon as we know the behaviour of the person, we can find the actual birthday money with the following query:

In [7]:
%%jetisu_query
select * from new_birthday_money where age = 10 and good_behaviour;

|age|good_behaviour|birthday_money|
|----|----|----|
|10.0|True|200.0|

And again to show that the direction of querying is flexible:

In [8]:
%%jetisu_query
select good_behaviour from new_birthday_money
    where age = 10
        and birthday_money = 200;

|good_behaviour|
|----|
|True|

# Test Data
The sigma complete relation lends itself to automated generation of test cases.  Here is an example of a test case that can be validated then copied and used with [pytest](https://docs.pytest.org/). The name of the function is generated by hashing the input query.

In [9]:
%%jetisu_testcase
select good_behaviour from new_birthday_money
    where age = 10
        and birthday_money = 200;

def test_idr_4cbd6b19e0():
    res = idr_query("""select good_behaviour from new_birthday_money
    where age = 10
        and birthday_money = 200;
""", 'data')
    assert idr_test_res_sort(res) == idr_test_res_sort((('good_behaviour',), [(True,)]))


## Next step
You can edit and re-run this example workbook by following the instructions here:
[How to run the example notebooks](https://github.com/DavidPratten/jetisu/blob/main/docs/run_notebooks.md)